# Make mel norm test data

This notebook reads in one of the golden data files for the mel matrix function test in `test_filters.py:test_melfb`, then writes several variants of it including an additional `norm` field (to be provided as a `norm=` argument to `filters.py:mel`) and writes out three new golden data files.  These are consumed by the new `__test_with_norm` private function within `test_filters.py:test_melfb`.

In [2]:
import os

import numpy as np
import librosa
import scipy.io

In [3]:
LIBROSA_TEST_PATH = '.'
infile = os.path.join(LIBROSA_TEST_PATH, 'data/feature-melfb-001.mat')
DATA = scipy.io.loadmat(infile, chars_as_strings=True)

In [4]:
print DATA

{'wts': array([[ 0.        ,  0.00492891,  0.00985782, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]]), 'frqs': array([[    0.        ,    79.62501637,   159.25003274,   238.87504912,
          318.50006549,   398.12508186,   477.75009823,   557.37511461,
          637.00013098,   716.62514735,   796.25016372,   875.8751801 ,
          955.50019647,  1036.88788871,  1125.62589636,  1221.95820045,
         1326.53472923,  1440.06103253,  1563.30304193,  1697.09223823,
         

In [5]:
DATA['norm'] = np.array([[1]])
print DATA['norm'].shape

(1, 1)


In [6]:
def melfbnorm_test(DATA):
    # if DATA['norm'] is empty, pass None.
    if DATA['norm'].shape[-1] == 0:
        norm = None
    else:
        norm = DATA['norm'][0][0]

    wts = librosa.filters.mel(DATA['sr'][0, 0],
                               DATA['nfft'][0, 0],
                               n_mels=DATA['nfilts'][0, 0],
                               fmin=DATA['fmin'][0, 0],
                               fmax=DATA['fmax'][0, 0],
                               htk=DATA['htk'][0, 0], 
                               # if DATA['norm'] is empty, pass None.
                               norm=norm)
    padded_wts = np.pad(wts, [(0, 0), (0, int(DATA['nfft'][0]//2) - 1)], mode='constant')
    return padded_wts

padded_wts = melfbnorm_test(DATA)


In [7]:
print padded_wts.shape
print DATA['wts'].shape

(20, 256)
(20, 256)


In [8]:
assert np.allclose(padded_wts, DATA['wts'])

In [9]:
TEST_NUM = 1

def save_next_test(data):
    global TEST_NUM
    outfile = os.path.join(LIBROSA_TEST_PATH, 'data/feature-melfbnorm-{:03d}.mat'.format(TEST_NUM))
    scipy.io.savemat(outfile, data)
    print 'wrote', outfile
    TEST_NUM += 1
    
save_next_test(DATA)

wrote ./data/feature-melfbnorm-001.mat


In [10]:
DATA['norm'] = np.array([[]])
print DATA['norm'].shape
padded_wts = melfbnorm_test(DATA)
# It's different from the last golden value.
assert not np.allclose(padded_wts, DATA['wts'])
# Make it the new golden value for this test.
DATA['wts'] = padded_wts

save_next_test(DATA)

(1, 0)
wrote ./data/feature-melfbnorm-002.mat


In [11]:
DATA['norm'] = np.array([[np.inf]])
padded_wts = melfbnorm_test(DATA)
# It's the same as the last golden value.
assert np.allclose(padded_wts, DATA['wts'])

save_next_test(DATA)

wrote ./data/feature-melfbnorm-003.mat
